# Section 602 - CONSTRUCTION CLASSIFICATION

## Building Code

### TABLE 602 FIRE-RESISTANCE RATING REQUIREMENTS FOR EXTERIOR WALLS BASED ON FIRE SEPARATION DISTANCEa, d, g 

In [13]:
import json 
# Load the table from the JSON file
with open('table_602.json', 'r') as f:
    table_602 = json.load(f)

In [14]:
import sys
import os

# Add the path to the Rule Notebooks folder to the system path
sys.path.append(os.path.abspath('../..'))

# Import the find_value function from table_lookup.py
from ruleFunctions import find_value, check_value, get_building_info, create_scrollable_table

In [19]:
# Define the SPARQL endpoint URL and query file path
endpoint_url = "http://localhost:3030/SmartReviewHotel/sparql"

# Call the function to get the building information
main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num = get_building_info(endpoint_url)
print(main_construction_type, subtype, occupancy_group, sprinkler_system, sprinkler_type, storey_num)

Type V A R-1 S 13 4


In [20]:
def get_range(distance):
    if distance < 5:
        return "< 5"
    elif 5 <= distance < 10:
        return "5 <= X < 10"
    elif 10 <= distance < 30:
        return "10 <= X < 30"
    else:
        return "<= 30"

def adjust_construction_type(distance, main_construction_type):
    # Adjust the construction type based on the distance and provided type
    adjusted_construction_type = main_construction_type
    if distance < 5 or distance > 30:
        adjusted_construction_type = "All"
    elif 5 <= distance < 10 and main_construction_type not in ["IA"]:
        adjusted_construction_type = "Others"
    elif 10 <= distance < 30 and main_construction_type not in ["IA", "IB", "IIB", "VB"]:
        adjusted_construction_type = "Others"
    return adjusted_construction_type

def get_value_from_lookup(distance, construction_type, occupancy_group):
    range_value = get_range(distance)
    for entry in table_602["FIRE_SEPARATION_DISTANCE"]:
        if entry["range"] == range_value and entry["TYPE_OF_CONSTRUCTION"] == construction_type:
            return entry.get(f"OCCUPANCY_GROUP_{occupancy_group}", None)
    return None

In [21]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(endpoint_url)
with open("..\..\..\SPARQL\exteriorNonbearing.sparql", "r") as file:
    sparql_query = file.read()

# Set the SPARQL query and the return format
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)

# Execute the query and get the results in JSON format
results = sparql.query().convert()

# Process SPARQL results
for result in results["results"]["bindings"]:
    fire_separation_distance = float(result["smrvExteriorFireSeparationDistance_property_simple"]["value"])
    main_construction_type = result["batid"]["value"]  # Assuming this is where the type is stored
    
    # Adjust construction type based on distance and provided type
    adjusted_construction_type = adjust_construction_type(fire_separation_distance, main_construction_type)
    
    occupancy_group = "A"  # Example occupancy group; adjust as needed
    
    # Get the value from lookup data
    value = get_value_from_lookup(fire_separation_distance, adjusted_construction_type, occupancy_group)
    
    print(f"Wall ID: {result['wall']['value']}")
    print(f"Fire Rating: {result['fireRating']['value']}")
    print(f"Fire Separation Distance: {fire_separation_distance} feet")
    print(f"Adjusted Construction Type: {adjusted_construction_type}")
    print(f"Value for Occupancy Group {occupancy_group}: {value}")
    print()  # Print a blank line for readability


In [22]:
print(results)

{'head': {'vars': ['wall', 'batid', 'fireRating', 'fireSeparationDistance']}, 'results': {'bindings': []}}
